In [1]:
import pandas as pd
import requests
import json

In [4]:
def fetch_daily_data(symbol):
    pair_split = symbol.split('/')  # symbol must be in format XXX/XXX ie. BTC/EUR
    symbol = pair_split[0] + '-' + pair_split[1]
    granularity = 60 # 86400
    url = f'https://api.pro.coinbase.com/products/{symbol}/candles?granularity={granularity}'
    response = requests.get(url)
    if response.status_code == 200:  # check to make sure the response from server is good
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
        data['vol_fiat'] = data['volume'] * data['close']      # multiply the BTC volume by closing price to approximate fiat volume

        # if we failed to get any data, print an error...otherwise write the file
        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            return data
            data.to_csv(f'Coinbase_{pair_split[0] + pair_split[1]}_dailydata.csv', index=False)
    else:
        print("Did not receieve OK response from Coinbase API")

In [5]:
pair = "BTC/USD"
data = fetch_daily_data(symbol=pair)

In [7]:
data

,unix,low,high,open,close,volume,date,vol_fiat
0,1613328240,48498.99,48599.33,48499.68,48549.52,15.251919,2021-02-14 18:44:00,7.404733e+05
1,1613328180,48480.01,48527.76,48527.76,48501.00,15.152758,2021-02-14 18:43:00,7.349239e+05
2,1613328120,48500.00,48543.72,48510.00,48527.76,27.730931,2021-02-14 18:42:00,1.345720e+06
3,1613328060,48509.71,48552.29,48511.90,48510.00,3.879654,2021-02-14 18:41:00,1.882020e+05
4,1613328000,48501.10,48525.46,48509.48,48511.89,11.347935,2021-02-14 18:40:00,5.505098e+05
...,...,...,...,...,...,...,...,...
295,1613310540,48713.50,48915.92,48915.92,48870.94,55.275528,2021-02-14 13:49:00,2.701367e+06
296,1613310480,48825.00,49019.95,49013.30,48915.22,30.825376,2021-02-14 13:48:00,1.507830e+06
297,1613310420,49011.56,49133.78,49129.54,49015.39,13.681719,2021-02-14 13:47:00,6.706148e+05
298,1613310360,49100.00,49135.56,49118.26,49129.68,4.244815,2021-02-14 13:46:00,2.085464e+05


In [2]:

# IMPORTS
import pandas as pd
import math
import os.path
import time
#from bitmex import bitmex
from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)

### API
#bitmex_api_key = '[REDACTED]'    #Enter your own API-key here
#bitmex_api_secret = '[REDACTED]' #Enter your own API-secret here
binance_api_key = 'O6wf2bZomPq7qM4XnF86WpAIGNQPtTh5LOcNrkhc3TlqqirbI6Nl8yfoevVTpVhN'
binance_api_secret = '47NDuVw5RxS15UmzBCk8XGGxlF5oLE8hwVsumXklr8AmTa8rfIQ2jEULT2Kzy5QF'

### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750
#bitmex_client = bitmex(test=False, api_key=bitmex_api_key, api_secret=bitmex_api_secret)
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)


### FUNCTIONS
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    #elif source == "bitmex": old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    #if source == "bitmex": new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): 
        print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: 
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['date'] = pd.to_datetime(data['timestamp'], unit='ms')
    
    # salva o appendi
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

def get_all_bitmex(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "bitmex")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    rounds = math.ceil(available_data / batch_size)
    if rounds > 0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in tqdm_notebook(range(rounds)):
            time.sleep(1)
            new_time = (oldest_point + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
            data = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            temp_df = pd.DataFrame(data)
            data_df = data_df.append(temp_df)
    data_df.set_index('timestamp', inplace=True)
    if save and rounds > 0: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df
#❤

In [18]:
symbol = 'BTCUSDT'
binance_symbols = ["BTCEUR", "XLMBTC"]
serie = []
for symbol in binance_symbols:
    serie.append(get_all_binance(symbol, kline_size='5m', save = True))

All caught up..!
All caught up..!


In [7]:
data.shape

(1758740, 11)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1758740 entries, 2017-12-08 06:01:00 to 2021-04-17 15:00:00
Data columns (total 11 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   open         object
 1   high         object
 2   low          object
 3   close        object
 4   volume       object
 5   close_time   int64 
 6   quote_av     object
 7   trades       int64 
 8   tb_base_av   object
 9   tb_quote_av  object
 10  ignore       object
dtypes: int64(2), object(9)
memory usage: 161.0+ MB


In [19]:
data5_xlm = pd.read_csv('XLMBTC-5m-data.csv')
data5_xlm['timestamp'] = pd.to_datetime(data_xlm['timestamp'])

data5_eur = pd.read_csv('BTCEUR-5m-data.csv')
data5_eur['timestamp'] = pd.to_datetime(data_eur['timestamp'])

In [24]:
data5_eur#['timestamp']

,timestamp,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
0,2020-01-03 08:00:00,6244.33,6478.91,6244.33,6478.91,0.130778,1578038699999,843.997189,4,0.066122,427.929327,0
1,2020-01-03 08:01:00,6478.91,6478.91,6478.91,6478.91,0.000000,1578038999999,0.000000,0,0.000000,0.000000,0
2,2020-01-03 08:02:00,6478.91,6478.91,6478.91,6478.91,0.000000,1578039299999,0.000000,0,0.000000,0.000000,0
3,2020-01-03 08:03:00,6478.91,6478.91,6478.91,6478.91,0.000000,1578039599999,0.000000,0,0.000000,0.000000,0
4,2020-01-03 08:04:00,6478.91,6478.91,6478.91,6478.91,0.000000,1578039899999,0.000000,0,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
135209,2020-04-06 14:31:00,50826.26,50845.62,50790.56,50845.62,3.411325,1618685999999,173363.318650,476,2.688100,136616.970112,0
135210,2020-04-06 14:32:00,50845.61,51015.38,50842.80,51001.70,7.803861,1618686299999,397726.556024,697,4.402690,224361.445455,0
135211,2020-04-06 14:33:00,51001.01,51065.60,50956.80,51051.06,6.775869,1618686599999,345688.654555,683,3.268526,166767.157284,0
135212,2020-04-06 14:34:00,51045.76,51100.00,51005.95,51100.00,3.431716,1618686899999,175214.989979,639,2.242465,114506.904783,0


In [17]:
data_xlm['timestamp']

0          2017-12-08 06:01:00
1          2017-12-08 06:02:00
2          2017-12-08 06:03:00
3          2017-12-08 06:04:00
4          2017-12-08 06:05:00
                  ...         
1758735    2021-04-17 14:56:00
1758736    2021-04-17 14:57:00
1758737    2021-04-17 14:58:00
1758738    2021-04-17 14:59:00
1758739    2021-04-17 15:00:00
Name: timestamp, Length: 1758740, dtype: object

In [2]:
1.26*58

73.08